In [1]:
import pandas as pd
import json
import time
from geopy.exc import GeocoderUnavailable, GeocoderTimedOut, GeocoderServiceError
from geopy.geocoders import Nominatim

rentdata_df = pd.read_json('rent_data.json')

In [2]:
title = []                  # 1
price = []                  # 2
area = []                   # 3
bedroom = []        # 4
description = []            # 5
width = []                  # 6
direction = []              # 7
toilet = []         # 8
address = []               # 9
geo_code = []               # 10
law = []
floor = []
district = []
province = []
type = []
type_estate = []
image_links = []
phonenumber = []

In [3]:
import re

# Add new column with default None values for geocodes
rentdata_df["geo_code"] = [None for _ in range(len(rentdata_df))]

def get_geocode(address):
    geolocator = Nominatim(user_agent="my_geocoder")
    
    # Retry parameters
    max_retries = 3
    retry_delay = 2  # seconds
    
    for attempt in range(max_retries):
        try:    
            location = geolocator.geocode(address + ", Vietnam", timeout=15)  # Add country information
            if location:
                return [location.latitude, location.longitude]
            else:
                # print(f"Error: Address not found for {address}!")
                return None
        except (GeocoderUnavailable, GeocoderTimedOut, GeocoderServiceError) as e:            
            print(f"Geocoder unavailable (attempt {attempt + 1}/{max_retries}). Retrying after {retry_delay} seconds.")
            time.sleep(retry_delay)
            retry_delay *= 2  # Exponential backoff
    return None

def data_normalization(df):
    for i, price in enumerate(df['price']):
        df.at[i, 'price'] = price.replace('Triệu', '').strip()
    
    for i, width in enumerate(df['width']):
        if isinstance(width, str):
            df.at[i, "width"] = width.replace('(m)', '').strip()
        
    for i, floor in enumerate(df['floor']):
        if isinstance(floor, str):
            df.at[i, "floor"] = floor.replace('(Tầng)', '').strip()
    
    for i, phonenumber in enumerate(df['phonenumber']):        
        if isinstance(phonenumber, str):
            phonenumber = phonenumber.replace(' ', '').replace('.', '').strip()
            # Kiểm tra nếu số đầu tiên khác 0 thì thêm số 0 vào đầu
            if phonenumber[0] != '0':
                phonenumber = '0' + phonenumber
            df.at[i, 'phonenumber'] = phonenumber.strip()
            
    for i, title in enumerate(df['title']):
        if isinstance(title, str):
            title = re.sub(r'[^\w\s,.()\–]', '', title)
            title = title.lower()
            title = title.capitalize()
            df.at[i, 'title'] = title.strip()    

data_normalization(rentdata_df)

if 'address' in rentdata_df.columns:
    for i, address in enumerate(rentdata_df["address"]):
        rentdata_df.at[i, "geo_code"] = get_geocode(address)
else:
    print("Error: 'address' column not found in the DataFrame.")

def write_data(df, field, data_list):
    if not isinstance(data_list, list):
        data_list = []  # Initialize as empty list if not already
    
    if field in df.columns:
        for value in df[field]:
            data_list.append(value)
    else:
        print(f"Error: '{field}' column not found in the DataFrame.")

write_data(rentdata_df, "title", title)
write_data(rentdata_df, "price", price)
write_data(rentdata_df, "area", area)
write_data(rentdata_df, "type", type)
write_data(rentdata_df, "type_estate", type_estate)
write_data(rentdata_df, "image_links", image_links)
write_data(rentdata_df, "bedroom", bedroom)
write_data(rentdata_df, "description", description)
write_data(rentdata_df, "width", width)
write_data(rentdata_df, "direction", direction)
write_data(rentdata_df, "toilet", toilet)
write_data(rentdata_df, "address", address)
write_data(rentdata_df, "law", law)
write_data(rentdata_df, "floor", floor)
write_data(rentdata_df, "geo_code", geo_code)
write_data(rentdata_df, "phonenumber", phonenumber)

# Display the updated DataFrame
print("\nUpdated DataFrame:")


Updated DataFrame:


In [4]:
rentdata_df

,title,price,area,type,address,description,law,type_estate,province,district,bedroom,toilet,floor,direction,width,image_links,phonenumber,geo_code
0,Cho thuê nhà mặt tiền nhìn phía đông nam ra hồ...,120,195.0,Cho Thuê,"Quảng An, Phường Quảng An, Quận Tây Hồ, Hà Nội",<p>Chính chủ cần cho thuê Nhà mặt phố tại Quản...,Có Sổ đỏ,Nhà mặt phố,Hà Nội,Tây Hồ,1.0,2.0,4,Đông nam,13,https://nhadat24h.net//Upload/User/DangTin/202...,784062792,"[21.0705937, 105.8299492]"
1,cho thuê nhà nguyên căn trong khu dân cư . gần...,11,100.0,Cho Thuê,"Phường Trường Thọ, Thành phố Thủ Đức, Hồ Chí Minh",None,None,Nhà mặt phố,Hồ Chí Minh,Thủ Đức,NaN,NaN,None,None,5,https://nhadat24h.net//Upload/User/avatar/2024...,378334550,"[10.836088, 106.7567252]"
2,cho thuê nhà nguyên căn trong khu dân cư . gần...,11,100.0,Cho Thuê,"Phường Bình Thọ, Thành phố Thủ Đức, Hồ Chí Minh",None,None,Nhà mặt phố,Hồ Chí Minh,Thủ Đức,NaN,NaN,None,None,5,https://nhadat24h.net//Upload/User/avatar/2024...,378334550,"[10.8456138, 106.7663383]"
3,cho thuê nhà nguyên căn trong khu dân cư . gần...,11,100.0,Cho Thuê,"Phường Linh Tây, Thành phố Thủ Đức, Hồ Chí Minh",- Cho THUÊ NHÀ NGUYÊN CĂN Trong khu dân cư . G...,None,Nhà mặt phố,Hồ Chí Minh,Thủ Đức,NaN,NaN,None,None,None,https://nhadat24h.net//Upload/User/avatar/2024...,378334550,"[10.857921, 106.754837]"
4,Cc cân cho thuê dai han căn goc 2 măt tiên đươ...,18.5,68.0,Cho Thuê,"Nguyễn Văn Tố, Phường Tân Thành, Quận Tân Phú,...",<p>Chính chủ cần cho thuê dài hạn căn nhà gó...,None,Nhà mặt phố,Hồ Chí Minh,Tân Phú,5.0,4.0,4,Tây,4,https://nhadat24h.net//Upload/User/DangTin/202...,934141927,"[10.7934021, 106.6332017]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,"Cho thuê toà nhà mặt tiền đường trần hưng đạo,...",95,990.0,Cho Thuê,"Phường An Phú, Quận Ninh Kiều, Cần Thơ","CHO THUÊ TOÀ NHÀ MẶT TIỀN ĐƯỜNG TRẦN HƯNG ĐẠO,...",None,Officetel,Cần Thơ,Ninh Kiều,NaN,NaN,None,None,None,https://nhadat24h.net//Upload/User/DangTin/202...,968203339,"[10.0322235, 105.778189]"
535,"Cho thuê mặt bằng văn phòng office cần thơ, đư...",15,50.0,Cho Thuê,"Phường An Hòa, Quận Ninh Kiều, Cần Thơ",CHO THUÊ MẶT BẰNG VĂN PHÒNG MẶT TIỀN ĐƯỜNG MẬU...,None,"Văn phòng, TTTM, Cửa hàng, Kiot",Cần Thơ,Ninh Kiều,1.0,1.0,10,None,15,https://nhadat24h.net//Upload/User/DangTin/202...,968203339,"[10.0459669, 105.7697904]"
536,"Cho thuê mặt bằng đường 3 tháng 2, gần đh cần ...",150,900.0,Cho Thuê,"Phường Hưng Lợi, Quận Ninh Kiều, Cần Thơ","<p dir=""auto"">________________________<br/>☎️T...",None,Nhà mặt phố,Cần Thơ,Ninh Kiều,1.0,1.0,1,Tây bắc,15,https://nhadat24h.net//Upload/User/avatar/2024...,968203339,"[10.0161234, 105.7639625]"
537,Khang thành invest chuyên cho thuê mini house...,7,35.0,Cho Thuê,"Phường Tân An, Quận Ninh Kiều, Cần Thơ",KHANG THÀNH Invest - CHUYÊN CHO THUÊ MINI HOUS...,None,Căn hộ chung cư,Cần Thơ,Ninh Kiều,1.0,1.0,3,None,10,https://nhadat24h.net//Upload/User/DangTin/202...,968203339,"[10.0318417, 105.785674]"


In [5]:
rentdata_df.to_csv('rent_data.csv', encoding='utf-8', index=False)


In [8]:
import pymysql

# Thiết lập kết nối tới cơ sở dữ liệu MySQL
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='123456',
    database='estate',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

try:
    with connection.cursor() as cursor:
        # Xóa dữ liệu từ cả hai bảng pools_bds và pools_image
        cursor.execute("""
            DELETE pb, pi
            FROM pools_bds pb
            JOIN pools_image pi ON pb.BDS_id = pi.BDS_id
            WHERE pb.kind = 'Cho Thuê' AND pi.content LIKE '%nhadat24h.net%'
        """)
        connection.commit()

finally:
    connection.close()


In [10]:
import csv
import pymysql

# Thiết lập kết nối tới cơ sở dữ liệu MySQL với mã hóa utf8mb4
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='123456',
    database='estate',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

def is_decimal(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

try:
    with connection.cursor() as cursor:
        
        # Đọc dữ liệu từ file CSV và nhập vào MySQL
        with open('rent_data.csv', 'r', encoding='utf-8') as file:
            csv_data = csv.DictReader(file)
            for row in csv_data:
                try:
                    district = row['district']
                    province_city = row['province']
                    
                    # Kiểm tra tồn tại address
                    cursor.execute("SELECT address_id FROM pools_address WHERE district = %s AND province_city = %s", (district, province_city))
                    existing_address = cursor.fetchone()
                    if existing_address:
                        address_id = existing_address['address_id']
                    else:
                        # Nếu không tìm thấy địa chỉ, bỏ qua dòng này
                        print(f"Không tìm thấy địa chỉ cho dòng: {row}")
                        continue
                    
                    # Kiểm tra pools_typeofbds 
                    bds_type = row['type_estate']
                    cursor.execute("SELECT Type_Id FROM pools_typeofbds WHERE TypeName = %s", (bds_type,))
                    existing_type = cursor.fetchone()
                    if existing_type:
                        type_id = existing_type['Type_Id']
                    else:
                        # Nếu không tìm thấy loại bất động sản, bỏ qua dòng này
                        print(f"Không tìm thấy loại bất động sản cho dòng: {row}")
                        continue
                    
                    # Thêm dữ liệu vào bảng pools_bds
                    title = row['title']
                    price = row['price']
                    area = row['area']
                    kind = row['type']
                    law = row['law']
                    bedroom = row['bedroom']
                    floor = row['floor']
                    phonenumber = row['phonenumber']
                    description = row['description']
                    width = row['width']
                    direction = row['direction']
                    toilet = row['toilet']
                    location = row['address']
                    geocode = row['geo_code']
                    
                    # Handle 'price' column
                    if price.strip().lower() == 'thỏa thuận':
                        price = 0  # or set to a default value, like 0
                    elif not is_decimal(price):
                        price = 0  # or set to a default value

                    # Handle 'width' column
                    if not is_decimal(width):
                        width = 0  # or set to a default value

                    # Handle 'bedroom' column
                    if not bedroom.strip() or bedroom.lower() == 'null':
                        bedroom = 0  # or set to a default value
                       
                    # Handle 'floor' column 
                    if not floor.strip() or floor.lower() == 'null':
                        floor = 0  # or set to a default value

                    # Handle 'toilet' column (if needed)
                    if not is_decimal(toilet):
                        toilet = 0  # or set to a default value

                    # Kiểm tra geocode nếu là None hoặc rỗng, bỏ qua dòng này
                    if not geocode:
                        # print(f"Không có giá trị geocode cho dòng: {row}")
                        continue

                    # Chuyển geo_code thành string nếu nó là list
                    if isinstance(geocode, list):
                        geocode = str(geocode)

                    # Thêm dữ liệu vào bảng pools_bds
                    cursor.execute("""
                        INSERT INTO pools_bds (title, price, area, kind, law, bedroom, floor, phonenumber, address_id, Type_id, description, width, direction, toilet, location, geocode)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    """, (title, price, area, kind, law, bedroom, floor, phonenumber, address_id, type_id, description, width, direction, toilet, location, geocode))
                    connection.commit()
                    
                    # Lấy BDS_id của BDS vừa thêm
                    cursor.execute("SELECT LAST_INSERT_ID()")
                    bds_id = cursor.fetchone()['LAST_INSERT_ID()']
                    
                    # Thêm dữ liệu vào bảng pools_image (nếu có)
                    if 'image_links' in row and row['image_links']:  # Kiểm tra xem cột 'image_link' có tồn tại và có giá trị không
                        image_link = row['image_links']
                        cursor.execute("INSERT INTO pools_image (BDS_id, content) VALUES (%s, %s)", (bds_id, image_link))
                        connection.commit()
                        
                except Exception as e:
                    print(f"Lỗi khi xử lý dòng: {row}")
                    print(e)
finally:
    connection.close()


Không tìm thấy loại bất động sản cho dòng: {'title': 'Cho thuê nhà 4,5 tầng vạn phúc hà đông dt75m2,giá 10tr đến 20trtháng', 'price': '20', 'area': '75.0', 'type': 'Cho Thuê', 'address': 'Kđt Vạn Phúc, Quận Hà Đông, Hà Nội', 'description': '<p></p><p>Chính chủ cho thuê tòa nhà văn phòng Tại Phố Vạn Phúc – Quận Hà Đông - TP Hà Nội</p><p>Tòa Nhà cao 7 Tầng , đường trước nhà 17m,nhà Hướng Nam,dt150m2,mặt tiền 10m</p><p>Diện tích sàn 150m2/1 sàn, thuê cả sàn 15tr/tháng lh 0936480475</p><p>Mỗi sàn được chia làm 3 phòng , mỗi phòng dt36m2,dt40m2,dt50m2, giá cho thuê từng phòng 4 triệu,5 triệu,6 triệu,có nhà vệ sinh khép kín từng phòng, có cầu thang máy,cầu thang bộ, vị trí đẹp giao thông đi lại thuận tiện,có nhà để xe riêng, cách ngã tư Vạn Phúc Tố Hữu Hà Đông 200m</p><p>Nhà 5 tầng liền kề vạn phúc hà đông,dt75m2,đường 17m,hướng ĐN,nhà đẹp giá rẻ!</p><p>Nhà 3,5 tầng lk5 vạn phúc hà đông,dt75m2,nhà hoàn thiện đẹp,giá 15tr/tháng</p><p>Nhà biệt thự DT165m2,mặt tiền 10m,đường 17m,hướng ĐN,giá ch